In [8]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from scipy.signal import argrelextrema
from backtesting import Strategy, Backtest

# Load data from CSV
# data = pd.read_csv("1h_TATAPOWER.csv")
# data['Datetime'] = pd.to_datetime(data['Datetime'])
# data.set_index('Datetime', inplace=True)

# Load data from CSV
data = pd.read_csv("D_RELIANCE1.csv")
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# data

# Calculate ATR
data['ATR'] = ta.atr(data.High, data.Low, data.Close, length=14)

# Find local maxima and minima
order = 7
max_idx = argrelextrema(data['High'].values, np.greater, order=order)[0]
min_idx = argrelextrema(data['Low'].values, np.less, order=order)[0]

# Create signal labels
labels = np.zeros(len(data))  # Initialize with non-pivot labels
labels[max_idx] = -1  # Set labels for pivot highs (sell signal)
labels[min_idx] = 1  # Set labels for pivot lows (buy signal)
data['Signal'] = labels

# data = pd.DataFrame()
# Define SIGNAL function
# data = data[1800:]
def SIGNAL():
    return data.Signal

# Define the strategy
class MyStrat(Strategy):
    mysize = 1
    # risk_reward_ratio = 5
    sl_coef = 0.01

    def init(self):
        self.signal = self.I(SIGNAL)

    def next(self):
        current_price = self.data.Close[-1]
        next_open = self.data.Close[-2]  # Next candle's open price
        sl_atr = self.sl_coef  * self.data.ATR[-1]
        sltr = self.sl_coef  * self.data.ATR[-1]

        if self.signal[-1] == 1:  # Buy signal
            if not self.position:
                entry_price = next_open + 0.1  # Example: entry 1 cent above open
                sl = entry_price  -  50
                tp = entry_price +   50*3 #sl_atr * 4
                self.buy(limit=entry_price, sl=sl, tp=tp, size=self.mysize)
            # elif self.position:
                # self.position.close()
                

        elif self.signal[-1] == -1:  # Sell signal
            # if not self.position:
            #     entry_price = next_open - 0.1  # Example: entry 1 cent above open
            #     sl = entry_price  + 50   #sl_atr / 1.2
            #     tp = entry_price  -  50*3 #sl_atr * 4
            #     self.sell(limit=entry_price, sl=sl, tp=tp, size=self.mysize)
            if self.position:
                self.position.close()

# data
data = data.dropna()


bt = Backtest(data, MyStrat, cash=1000, margin=1/5, commission=0.05)
stats, heatmap = bt.optimize(
    sl_coef=tuple(np.linspace(1, 2, 10)),  # Convert NumPy array to tuple
    # n1 = range(1,2,10),
    # n2 = range(1,10,100),
    # constraint = lambda x : x.n2 - x.n1 > 10,
    maximize='Return [%]',
    random_state=0,
    return_heatmap=True)

print(stats)
bt.plot()

# colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# plt.figure(figsize=(12, 6))
# plt.plot(data['Close'], zorder=0)
# plt.scatter(data.iloc[max_idx].index, data.iloc[max_idx]['Close'],
#   label='Maxima', s=50, color=colors[1], marker='v')
# plt.scatter(data.iloc[min_idx].index, data.iloc[min_idx]['Close'],
#   label='Minima', s=50, color=colors[2], marker='^')

# plt.legend()
# plt.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_4944\1101622592.py:77: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, MyStrat, cash=1000, margin=1/5, commission=0.05)
C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\backtesting\backtesting.py:912: UserWarning: (2024-01-24 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\backtesting\backtesting.py:912: UserWarning: (2024-02-29 00:00:00) A contingent SL/TP o

  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\backtesting\backtesting.py:912: UserWarning: (2024-01-24 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\backtesting\backtesting.py:912: UserWarning: (2024-02-29 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
 

Start                     2023-01-20 00:00:00
End                       2024-06-05 00:00:00
Duration                    502 days 00:00:00
Exposure Time [%]                   30.149254
Equity Final [$]                   569.486753
Equity Peak [$]                   1039.275891
Return [%]                         -43.051325
Buy & Hold Return [%]               26.033171
Return (Ann.) [%]                   -34.52654
Volatility (Ann.) [%]               38.133861
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -46.661502
Avg. Drawdown [%]                  -31.596101
Max. Drawdown Duration      307 days 00:00:00
Avg. Drawdown Duration      178 days 00:00:00
# Trades                                   12
Win Rate [%]                        41.666667
Best Trade [%]                       4.079686
Worst Trade [%]                     -4.761905
Avg. Trade [%]                    

GridPlot(id='p1351', ...)

In [3]:
pip install pandas_ta

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas_ta-0.3.14b.tar.gz (115 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218928 sha256=8c01104fac6e690fb2da8c17401efbe3a4dd29abb36660cbe5b4d3f4402552ba
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\fd\ed\18\2a12fd1b7906c63efca6accb351929f2c7f6bbc674e1c0ba5d
Successfully built pandas_ta
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip show pandas_ta

Name: pandas_ta
Version: 0.3.14b0
Summary: An easy to use Python 3 Pandas Extension with 130+ Technical Analysis Indicators. Can be called from a Pandas DataFrame or standalone like TA-Lib. Correlation tested with TA-Lib.
Home-page: https://github.com/twopirllc/pandas-ta
Author: Kevin Johnson
Author-email: appliedmathkj@gmail.com
License: The MIT License (MIT)
Location: C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages
Requires: pandas
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd 
import numpy as np
from backtesting.test import SMA

# data = pd.read_csv("1h_TATAPOWER.csv")
data = pd.read_csv("D_RELIANCE1.csv")

# data['Date'] = pd.to_datetime(data['Date'])
# data.set_index('Date', inplace=True)

data = data.dropna()
# data = pd.DataFrame()

# data= data[1800:2400]
class SmaCross(Strategy):
    n1 = 5 #10
    n2 = 10 #20
    mysize = 30
    sl_coef = 10

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):

         # get previous bar close
        price = self.data["Close"][-1]
    

        # long/short SL and TP
        long_sl = price * 0.98
        long_tp = price * 1.03
        short_sl = price * 1.02
        short_tp = price * 0.97

        # signal logic
        if crossover(self.sma1, self.sma2):
                self.buy(sl = long_sl, tp = long_tp,size=self.mysize)
        elif crossover(self.sma2, self.sma1):
            self.sell(sl = short_sl, tp = short_tp, size=self.mysize)

        # if crossover(self.sma1, self.sma2):
        #     if not self.position:
        #         self.buy(sl = long_sl, tp = long_tp,size=self.mysize)
        #         # self.cross_price = self.data.Close[-1]
        #         # self.buy(size=self.mysize)
        #     # elif self.position:
        #          # self.position.close()
        # elif crossover(self.sma2, self.sma1):
        #     if self.position:
        #     # # self.cross_price = self.data.Close[-1]
        #     #     self.sell(size=self.mysize)
        #     # elif self.position:
        #          self.position.close()

        
        # tsl
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, 
                            self.data["Close"][-1] * 0.98) 
            else: # short
                trade.sl = min(trade.sl or np.inf, 
                            self.data["Close"][-1] * 1.02) 

        
# bt = Backtest(GOOG, SmaCross,
#               cash=10000, commission=.002,
#               exclusive_orders=True)


bt = Backtest(data, SmaCross, cash=100000, commission=.002)
stats = bt.run()

print(stats)
bt.plot()

In [4]:
import yfinance as yf

ticker = 'TATAPOWER.NS'
data = yf.download(ticker,start='2024-05-20', end='2024-6-20',interval='5m')
# data.to_csv("5m_TATAPOWER.csv")

ModuleNotFoundError: No module named 'yfinance'